In [ ]:
import json

notebook = {
    "cells": [
        {
            "cell_type": "markdown",
            "metadata": {},
            "source": ["# TP3 Python\n\nEste notebook contém a resolução das questões do TP3."]
        },
        {
            "cell_type": "code",
            "metadata": {},
            "source": ["import pandas as pd\nfrom sqlalchemy import create_engine, Table, Column, Integer, String, MetaData"]
        },
        {
            "cell_type": "markdown",
            "metadata": {},
            "source": ["## Questão 1"]
        },
        {
            "cell_type": "code",
            "metadata": {},
            "source": [
                "# Caminho do arquivo (substitua com o link do Github se necessário)\n",
                "file_path = 'https://github.com/SrHya/TP3_Python/raw/main/car_crashes.xlsx'\n",
                "\n",
                "# Carregar o arquivo Excel em um DataFrame\n",
                "car_crashes_df = pd.read_excel(file_path)\n",
                "\n",
                "# Apresentar o DataFrame\n",
                "car_crashes_df.head()"
            ]
        },
        {
            "cell_type": "markdown",
            "metadata": {},
            "source": ["## Questão 2"]
        },
        {
            "cell_type": "code",
            "metadata": {},
            "source": [
                "# Agrupar os dados pela variável 'ins_type' e calcular as métricas solicitadas\n",
                "grouped_df = car_crashes_df.groupby('ins_type').agg({\n",
                "    'ins_premium': ['mean', 'max', 'min'],\n",
                "    'ins_losses': ['mean', 'sum', 'max', 'min']\n",
                "})\n",
                "\n",
                "# Apresentar o DataFrame resultante\n",
                "grouped_df"
            ]
        },
        {
            "cell_type": "markdown",
            "metadata": {},
            "source": ["## Questão 3"]
        },
        {
            "cell_type": "code",
            "metadata": {},
            "source": [
                "# Caminho do arquivo (substitua com o link do Github se necessário)\n",
                "db_url = 'https://github.com/SrHya/TP3_Python/raw/main/mpg.db'\n",
                "\n",
                "# Para simular o download do arquivo para o Colab, você pode usar o seguinte comando:\n",
                "# !wget -O /content/mpg.db {db_url}\n",
                "\n",
                "# Criar a conexão com a base de dados SQLite\n",
                "engine = create_engine('sqlite:///mpg.db')\n",
                "\n",
                "# Identificar as tabelas na base de dados\n",
                "table_names_query = \"SELECT name FROM sqlite_master WHERE type='table';\"\n",
                "table_names = pd.read_sql_query(table_names_query, engine)\n",
                "\n",
                "# Carregar cada tabela em um DataFrame próprio\n",
                "observation_df = pd.read_sql_table('Observation', engine)\n",
                "origin_df = pd.read_sql_table('Origin', engine)\n",
                "name_df = pd.read_sql_table('Name', engine)\n",
                "\n",
                "# Exibir as tabelas carregadas\n",
                "observation_df.head(), origin_df.head(), name_df.head()"
            ]
        },
        {
            "cell_type": "markdown",
            "metadata": {},
            "source": ["## Questão 4"]
        },
        {
            "cell_type": "code",
            "metadata": {},
            "source": [
                "# Conectar ao banco de dados SQLite\n",
                "connection = engine.connect()\n",
                "metadata = MetaData()\n",
                "\n",
                "# Definir a nova tabela 'Category'\n",
                "category_table = Table('Category', metadata,\n",
                "                       Column('category_id', Integer, primary_key=True),\n",
                "                       Column('category', String))\n",
                "\n",
                "# Criar a tabela no banco de dados\n",
                "metadata.create_all(engine)\n",
                "\n",
                "# Inserir um registro na nova tabela\n",
                "insert_query = category_table.insert().values(category_id=0, category='sedan')\n",
                "connection.execute(insert_query)\n",
                "\n",
                "# Verificar se a tabela foi criada e o registro inserido\n",
                "category_df = pd.read_sql_table('Category', engine)\n",
                "category_df"
            ]
        },
        {
            "cell_type": "markdown",
            "metadata": {},
            "source": ["## Questão 5"]
        },
        {
            "cell_type": "code",
            "metadata": {},
            "source": [
                "# Mesclar os DataFrames 'Observation', 'Origin' e 'Name'\n",
                "merged_df = observation_df.merge(origin_df, on='origin_id').merge(name_df, on='name_id')\n",
                "\n",
                "# Exibir o DataFrame agregado\n",
                "merged_df.head()"
            ]
        },
        {
            "cell_type": "markdown",
            "metadata": {},
            "source": ["## Questão 6"]
        },
        {
            "cell_type": "code",
            "metadata": {},
            "source": [
                "# Remover as colunas 'name_id' e 'origin_id' do DataFrame agregado\n",
                "merged_df.drop(columns=['name_id', 'origin_id'], inplace=True)\n",
                "\n",
                "# Exibir o DataFrame resultante\n",
                "merged_df.head()"
            ]
        },
        {
            "cell_type": "markdown",
            "metadata": {},
            "source": ["## Questão 7"]
        },
        {
            "cell_type": "code",
            "metadata": {},
            "source": [
                "# Calcular o consumo médio (mpg) por país de origem\n",
                "mpg_by_origin = merged_df.groupby('origin')['mpg'].mean().reset_index()\n",
                "\n",
                "# País de origem dos carros que mais consomem combustível (menor mpg)\n",
                "most_fuel_consuming_origin = mpg_by_origin.loc[mpg_by_origin['mpg'].idxmin()]\n",
                "\n",
                "# Exibir os resultados\n",
                "mpg_by_origin, most_fuel_consuming_origin"
            ]
        },
        {
            "cell_type": "markdown",
            "metadata": {},
            "source": ["## Questão 8"]
        },
        {
            "cell_type": "code",
            "metadata": {},
            "source": [
                "# Calcular o peso médio dos carros por país e por quantidade de cilindros\n",
                "weight_by_origin_cylinders = merged_df.pivot_table(\n",
                "    values='weight', \n",
                "    index='cylinders', \n",
                "    columns='origin', \n",
                "    aggfunc='mean'\n",
                ").reset_index()\n",
                "\n",
                "# Exibir o resultado\n",
                "weight_by_origin_cylinders"
            ]
        },
        {
            "cell_type": "markdown",
            "metadata": {},
            "source": ["## Questão 9"]
        },
        {
            "cell_type": "code",
            "metadata": {},
            "source": [
                "# Calcular a mediana da potência (horsepower) por ano de fabricação (model_year) e por quantidade de cilindros (cylinders)\n",
                "horsepower_median = merged_df.pivot_table(\n",
                "    values='horsepower',\n",
                "    index='model_year',\n",
                "    columns='cylinders',\n",
                "    aggfunc='median',\n",
                "    fill_value=0\n",
                ").reset_index()\n",
                "\n",
                "# Exibir o resultado\n",
                "horsepower_median"
            ]
        },
        {
            "cell_type": "markdown",
            "metadata": {},
            "source": ["## Questão 10"]
        },
        {
            "cell_type": "code",
            "metadata": {},
            "source": &#8203;:citation[oaicite:0]{index=0}&#8203;
